# Chapter 2: Emitting Basic MLIR

Now that we're familiar with our language and the AST, let's see how MLIR can
help to compile Toy.

## Introduction: Multi-Level Intermediate Representation

Other compilers, like LLVM (see the
[Kaleidoscope tutorial](https://llvm.org/docs/tutorial/MyFirstLanguageFrontend/index.html)),
offer a fixed set of predefined types and (usually *low-level* / RISC-like)
instructions. It is up to the frontend for a given language to perform any
language-specific type-checking, analysis, or transformation before emitting
LLVM IR. For example, Clang will use its AST to perform not only static analysis
but also transformations, such as C++ template instantiation through AST cloning
and rewrite. Finally, languages with construction at a higher-level than C/C++
may require non-trivial lowering from their AST to generate LLVM IR.

As a consequence, multiple frontends end up reimplementing significant pieces of
infrastructure to support the need for these analyses and transformation. MLIR
addresses this issue by being designed for extensibility. As such, there are few
pre-defined instructions (*operations* in MLIR terminology) or types.

## Interfacing with MLIR

MLIR is designed to be a completely extensible infrastructure; there is no
closed set of attributes (think: constant metadata), operations, or types. MLIR
supports this extensibility with the concept of Dialects. Dialects provide a grouping 
mechanism for abstraction under a unique `namespace`.

In MLIR, `Operations` are the core unit of abstraction and computation, similar in many 
ways to LLVM instructions. Operations can have application-specific semantics and can be 
used to represent all of the core IR structures in LLVM: instructions, globals (like 
functions), modules, etc.

Here is the MLIR assembly for the Toy `transpose` operations:

```mlir
%t_tensor = "toy.transpose"(%tensor) {inplace = true} : (tensor<2x3xf64>) -> tensor<3x2xf64> loc("example/file/path":12:1)
```

Let's break down the anatomy of this MLIR operation:

-   `%t_tensor`

    *   The name given to the result defined by this operation (which includes
        [a prefixed sigil to avoid collisions](../../LangRef.md/#identifiers-and-keywords)).
        An operation may define zero or more results (in the context of Toy, we
        will limit ourselves to single-result operations), which are SSA values.
        The name is used during parsing but is not persistent (e.g., it is not
        tracked in the in-memory representation of the SSA value).

-   `"toy.transpose"`

    *   The name of the operation. It is expected to be a unique string, with
        the namespace of the dialect prefixed before the "`.`". This can be read
        as the `transpose` operation in the `toy` dialect.

-   `(%tensor)`

    *   A list of zero or more input operands (or arguments), which are SSA
        values defined by other operations or referring to block arguments.

-   `{ inplace = true }`

    *   A dictionary of zero or more attributes, which are special operands that
        are always constant. Here we define a boolean attribute named 'inplace'
        that has a constant value of true.

-   `(tensor<2x3xf64>) -> tensor<3x2xf64>`

    *   This refers to the type of the operation in a functional form, spelling
        the types of the arguments in parentheses and the type of the return
        values afterward.

-   `loc("example/file/path":12:1)`

    *   This is the location in the source code from which this operation
        originated.

Shown here is the general form of an operation. As described above,
the set of operations in MLIR is extensible. Operations are modeled
using a small set of concepts, enabling operations to be reasoned
about and manipulated generically. These concepts are:

-   A name for the operation.
-   A list of SSA operand values.
-   A list of attributes.
-   A list of types for result values.
-   A source location for debugging purposes.
-   A list of successors blocks (for branches, mostly).
-   A list of regions (for structural operations like functions).

In MLIR, every operation has a mandatory source location associated with it.
Contrary to LLVM, where debug info locations are metadata and can be dropped, in
MLIR, the location is a core requirement, and APIs depend on and manipulate it.
Dropping a location is thus an explicit choice which cannot happen by mistake.

To provide an illustration: If a transformation replaces an operation by
another, that new operation must still have a location attached. This makes it
possible to track where that operation came from.

It's worth noting that the mlir-opt tool - a tool for testing
compiler passes - does not include locations in the output by default. The
`-mlir-print-debuginfo` flag specifies to include locations. (Run `mlir-opt
--help` for more options.)

### Opaque API

MLIR is designed to allow all IR elements, such as attributes, operations, and
types, to be customized. At the same time, IR elements can always be reduced to
the above fundamental concepts. This allows MLIR to parse, represent, and
round-trip IR for *any*
operation. For example, we could place our Toy operation from above into an
`.mlir` file and round-trip through *mlir-opt* without registering any `toy`
related dialect:

```mlir
func @toy_func(%tensor: tensor<2x3xf64>) -> tensor<3x2xf64> {
  %t_tensor = "toy.transpose"(%tensor) { inplace = true } : (tensor<2x3xf64>) -> tensor<3x2xf64>
  return %t_tensor : tensor<3x2xf64>
}
```

In the cases of unregistered attributes, operations, and types, MLIR will
enforce some structural constraints (e.g. dominance, etc.), but otherwise they
are completely opaque. For instance, MLIR has little information about whether
an unregistered operation can operate on particular data types, how many
operands it can take, or how many results it produces. This flexibility can be
useful for bootstrapping purposes, but it is generally advised against in mature
systems. Unregistered operations must be treated conservatively by
transformations and analyses, and they are much harder to construct and
manipulate.

This handling can be observed by crafting what should be an invalid IR for Toy
and seeing it round-trip without tripping the verifier:

```mlir
func @main() {
  %0 = "toy.print"() : () -> tensor<2x3xf64>
}
```

There are multiple problems here: the `toy.print` operation is not a terminator;
it should take an operand; and it shouldn't return any values. In the next
section, we will register our dialect and operations with MLIR, plug into the
verifier, and add nicer APIs to manipulate our operations.


## Defining Toy Operations

Now that we have a `Toy` dialect, we can start defining the operations. This
will allow for providing semantic information that the rest of the system can
hook into. As an example, let's walk through the creation of a `toy.constant`
operation. This operation will represent a constant value in the Toy language.

```mlir
 %4 = "toy.constant"() {value = dense<1.0> : tensor<2x3xf64>} : () -> tensor<2x3xf64>
```

This operation takes zero operands, a dense elements attribute named `value` 
to represent the constant value, and returns a single result of RankedTensorType. 
Let's take a look at the full definition and step through it in detail.


In [1]:
from typing import Annotated

from xdsl.dialects.builtin import TensorType, f64, DenseIntOrFPElementsAttr
from xdsl.irdl import IRDLOperation, OpAttr, irdl_op_definition, OpResult
from xdsl.utils.exceptions import VerifyException

from toy.dialects import toy

@irdl_op_definition
class ConstantOp(IRDLOperation):
    '''
    Constant operation turns a literal into an SSA value. The data is attached
    to the operation as an attribute. For example:

      %0 = toy.constant dense<[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]>
                        : tensor<2x3xf64>
    '''
    name: str = "toy.constant"
    value: OpAttr[DenseIntOrFPElementsAttr]
    res: Annotated[OpResult, toy.TensorTypeF64]

    @staticmethod
    def from_list(data: list[float], shape: list[int]):
        value = DenseIntOrFPElementsAttr.tensor_from_list(data, f64, shape)

        return ConstantOp.create(result_types=[value.type],
                                 attributes={"value": value})

    def verify_(self) -> None:
        resultType = self.res.typ
        value = self.value
        if not isinstance(resultType, TensorType):
            raise VerifyException("Expected result type to be `TensorTypeF64`")

        if not isinstance(value, DenseIntOrFPElementsAttr):
            raise VerifyException(
                "Expected value type to be instance of `DenseIntOrFPElementsAttr`"
            )

        if resultType.get_shape() != value.shape:
            raise VerifyException(
                "Expected value and result to have the same shape")

### The Operation base class

Each operation subclasses the `Operation` class, and are prefixed by the 
`@irdl_op_definition` decorator. The decorator steps through the definition of the
operation, and connverts it into a canonical form that the compilation engine can use.
Each operation must have a `name` of type `str`, and the name must be composed of the name
of the dialect, in this case `toy`, followed by a unique name of the operation. The name
is followed by the fields of the operation, along with their constraints.

### Operation fields

An operation is composed of operands, attributes, and results. In the case of the constant
operation, there is no input, a single attribute containing the initial value for the
variable, and a result containing the SSA value of the variable itself. Here is the format
for operation field definitions:

``` python
@irdl_op_definition
class MyOp(Operation):
  name: 'my_dialect.my_op'

  # Normal attribute field, conforming to MyAttributeConstraint
  my_attribute: OpAttr[MyAttributeConstraint] 
  # Optional attribute field
  optional_attribute: OptOpAttr[MyAttributeConstraint]
  # Variadic attribute field
  variadic_attribute: VarOpAttr[MyAttributeConstraint]

  # Normal operand field, conforming to MyOperandConstraint
  my_operand: Annotated[Operand, MyOperandConstraint] 
  # Optional operand field
  optional_attribute: Annotated[OptOperand, MyOperandConstraint]
  # Variadic operand field
  variadic_attribute: Annotated[VarOperand, MyOperandConstraint]

  # Normal result field, conforming to MyResultConstraint
  my_result: Annotated[OpResult, MyResultConstraint]
  # Optional result field
  optional_result: Annotated[OptOpResult, MyResultConstraint]
  # Variadic result field
  variadic_result: Annotated[VarOpResult, MyResultConstraint]
```

Please see `dialect.py` for the definitions of other operations in the dialect, which
use these constructs.

### Constructor helper

```
    @staticmethod
    def from_list(data: list[float], shape: list[int]):
        value = DenseIntOrFPElementsAttr.tensor_from_list(data, f64, shape)

        return ConstantOp.create(result_types=[value.type],
                                 attributes={"value": value})
```

Operations tend to have helper methods for constructing them, that call into the generic
constructors on the Operation class. In this case, the client passes in a flat python
`list` of `float`s for the data, and a shape definition. These get converted to the
attribute and result type that the `create` method expects as input.

### Custom verifier

``` python
    def verify_(self) -> None:
        resultType = self.res.typ
        value = self.value
        if not isinstance(resultType, TensorType):
            raise VerifyException("Expected result type to be `TensorTypeF64`")

        if not isinstance(value, DenseIntOrFPElementsAttr):
            raise VerifyException(
                "Expected value type to be instance of `DenseIntOrFPElementsAttr`"
            )

        if resultType.get_shape() != value.shape:
            raise VerifyException(
                "Expected value and result to have the same shape")
```

One thing to notice here is that all of our Toy operations are printed using the
generic assembly format. This format is the one shown when breaking down
`toy.transpose` at the beginning of this chapter. MLIR allows for operations to
define their own custom assembly format, either or imperatively via C++. Defining a custom 
assembly format allows for tailoring the generated IR into something a bit more readable 
by removing a lot of the fluff that is required by the generic format. Let's walk through 
an example of an operation format that we would like to simplify.

This capability will soon be added to xDSL also, and will be interoperable with the MLIR
format definitions.


In [2]:
from xdsl.ir import MLContext
from xdsl.printer import Printer

# MLContext, containing information about the registered dialects
ctx = MLContext()

ctx.register_dialect(toy.Toy)

printer = Printer()

In [3]:
from xdsl.ir import SSAValue, BlockArgument, OpResult
from xdsl.dialects.builtin import ModuleOp, f64, FunctionType
from xdsl.builder import Builder


@ModuleOp
@Builder.implicit_region
def module_op():
    unrankedf64TensorType = toy.UnrankedTensorType.from_type(f64)

    multiply_transpose_type = FunctionType.from_lists(
        [unrankedf64TensorType, unrankedf64TensorType],
        [unrankedf64TensorType])

    @Builder.implicit_region(multiply_transpose_type.inputs)
    def multiply_transpose(args: tuple[BlockArgument, ...]) -> None:
        a, b = args
        a_t = toy.TransposeOp(a).res
        b_t = toy.TransposeOp(b).res
        prod = toy.MulOp(a_t, b_t).res
        toy.ReturnOp(prod)

    def call_multiply_transpose(a: SSAValue, b: SSAValue) -> OpResult:
        return toy.GenericCallOp("multiply_transpose", [a, b],
                                 [unrankedf64TensorType]).res[0]

    main_type = FunctionType.from_lists([], [])

    @Builder.implicit_region
    def main() -> None:
        a = toy.ConstantOp.from_list([1, 2, 3, 4, 5, 6], [2, 3]).res
        b_0 = toy.ConstantOp.from_list([1, 2, 3, 4, 5, 6], [6]).res
        b = toy.ReshapeOp(b_0, [2, 3]).res
        c = call_multiply_transpose(a, b)
        call_multiply_transpose(b, a)
        call_multiply_transpose(b, c)
        a_t = toy.TransposeOp(a).res
        call_multiply_transpose(a_t, c)
        toy.ReturnOp()

    toy.FuncOp("multiply_transpose",
               multiply_transpose_type,
               multiply_transpose,
               private=True)
    toy.FuncOp("main", main_type, main)


printer.print(module_op)


"builtin.module"() ({
  "toy.func"() ({
  ^0(%0 : tensor<*xf64>, %1 : tensor<*xf64>):
    %2 = "toy.transpose"(%0) : (tensor<*xf64>) -> tensor<*xf64>
    %3 = "toy.transpose"(%1) : (tensor<*xf64>) -> tensor<*xf64>
    %4 = "toy.mul"(%2, %3) : (tensor<*xf64>, tensor<*xf64>) -> tensor<*xf64>
    "toy.return"(%4) : (tensor<*xf64>) -> ()
  }) {"sym_name" = "multiply_transpose", "function_type" = (tensor<*xf64>, tensor<*xf64>) -> tensor<*xf64>, "sym_visibility" = "private"} : () -> ()
  "toy.func"() ({
    %5 = "toy.constant"() {"value" = dense<[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]> : tensor<2x3xf64>} : () -> tensor<2x3xf64>
    %6 = "toy.constant"() {"value" = dense<[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]> : tensor<6xf64>} : () -> tensor<6xf64>
    %7 = "toy.reshape"(%6) : (tensor<6xf64>) -> tensor<2x3xf64>
    %8 = "toy.generic_call"(%5, %7) {"callee" = @multiply_transpose} : (tensor<2x3xf64>, tensor<2x3xf64>) -> tensor<*xf64>
    %9 = "toy.generic_call"(%7, %5) {"callee" = @multiply_transpose} : (te